In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [97]:
response = requests.get('https://www.bls.gov/opub/ted/2021/job-gains-and-losses-by-state-from-march-2019-to-march-2021.htm')
html_text = response.text
data = BeautifulSoup(html_text, 'html.parser')

def scrape_table(i):
    tables = data.find_all(class_ = 'article-table')
    table_data = []
    
    for row in tables[i].find_all('tr'):
        row_list = []
        for value in row.stripped_strings:
            row_list.append(value)
        table_data.append(row_list)
        
    df = pd.DataFrame(data = table_data[1:], 
                      columns = table_data[0])
    df.set_index('State', inplace = True)
    df.dropna(how='all', inplace=True)
    return df

scrape_table(0).to_csv("./Data/BLS state job gains and losses/Gross_job_gains.csv")
scrape_table(1).to_csv("./Data/BLS state job gains and losses/Gross_job_losses.csv")

### Extract data for unemployment rates for States in the USA from bls.gov website

In [14]:
def scrape_yearly_rate(year_id):
    response = requests.get('https://www.bls.gov/lau/'+ year_id + '.htm')
    html_text = response.text
    data = BeautifulSoup(html_text, 'html.parser')
    table = data.find(id = year_id)

    data = []
    for row in table.find_all('tr'):
        row_list = []
        for value in row.stripped_strings:
            row_list.append(value)
        row_list.append('20' + year_id[6:])
        data.append(row_list)
    df = pd.DataFrame(data = data[3:-1],
                      columns=['State', 'Rate', 'Rank', 'Year'])
    return df

In [15]:
id_list = ['lastrk20', 'lastrk19', 'lastrk18', 'lastrk17', 'lastrk16']
for id in id_list:
    scrape_yearly_rate(id).to_csv("./Data/Unemployment Rates for States/year_20" + id[6:] + ".csv")

In [3]:
response = requests.get('https://www.bls.gov/charts/state-employment-and-unemployment/state-unemployment-rates-animated.htm')
html_text = response.text
data = BeautifulSoup(html_text, 'html.parser')

table = data.find(id = 'lau_rc_unmapanim')
table_data = []
for row in table.find_all('tr'):
    row_list = []
    for value in row.stripped_strings:
        row_list.append(value)
    table_data.append(row_list)
df = pd.DataFrame(data = table_data[1:], 
                      columns = table_data[0])
df.set_index('State', inplace=True)
df.to_csv('./Data/Unemployment Rates for States/state_unemployment_11_21.csv')